In [1]:
#integrasi web
from tensorflow.keras.models import load_model
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import tensorflow as tf
from IPython.display import Image as IPImage, display
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# Fungsi untuk mengunduh dan membaca gambar dari URL
def load_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')
    return img

# Fungsi untuk mengekstraksi URL gambar dari halaman web
def extract_image_urls(url):
    image_urls = []
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for img in soup.find_all('img'):
            img_url = img.get('src')
            if img_url:
                # Convert relative URL to absolute URL
                img_url = urljoin(url, img_url)
                
                # Ensure the URL is valid and well-formed
                parsed_url = urlparse(img_url)
                if parsed_url.scheme in ['http', 'https']:
                    image_urls.append(img_url)
    except Exception as e:
        print("Error:", e)
    return image_urls

# Fungsi untuk menampilkan gambar dan prediksi
def display_and_predict_images(image_urls, model):
    predictions = []
    
    for img_url in image_urls:
        try:
            img = load_image_from_url(img_url)
            img_array = np.array(img)
            
            # Preprocess image for the model
            resize = tf.image.resize(img_array, (256, 256))
            img_expanded = np.expand_dims(resize / 255.0, axis=0)  # Normalize and add batch dimension
            
            # Make prediction
            yhat = model.predict(img_expanded)
            class_name = 'Non Judi' if yhat > 0.5 else 'Judi'
            
            # Append to predictions list
            predictions.append((img_url, class_name, yhat))
            
        except Exception as e:
            print(f"Error processing image {img_url}: {e}")

    # Sort predictions to display 'Judi' first
    predictions.sort(key=lambda x: x[1])  # Sort by class name ('Judi' will come before 'Non Judi')

    for img_url, class_name, yhat in predictions:
        try:
            display(IPImage(url=img_url))
            print(f'Predicted class: {class_name}')
        except Exception as e:
            print(f"Error displaying image {img_url}: {e}")

# URL dari website yang ingin Anda ambil gambar-gambarnya
website_url = "https://marssihsaan.github.io/webtesting/"

# Mengambil URL gambar dari website
image_urls = extract_image_urls(website_url)

# Muat model h5 Anda
model = load_model('models/modelbaru8.h5')  # Corrected path

# Menampilkan gambar-gambar dan prediksi
display_and_predict_images(image_urls, model)





1/1 [==============================] - 0s 31ms/step


Predicted class: Judi


Predicted class: Judi


Predicted class: Judi


Predicted class: Non Judi


Predicted class: Non Judi


Predicted class: Non Judi


Predicted class: Non Judi


Predicted class: Non Judi
